<a href="https://colab.research.google.com/github/Rok-sana/ML-course/blob/main/Homework_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

1.**NER**

 Loading the pretrained tokenizers and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Creating a pipeline using the pretrained tokenizer and model

In [ ]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
example = "My name is Tim Cook, I'm the CEO of Apple. Apple has it's headquarters in the US."

Loading the pretrained tokenizers and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

In [ ]:
ner_results = nlp(example)

In [ ]:
for item in ner_results:
  print(item)

{'entity': 'B-PER', 'score': 0.99968755, 'index': 4, 'word': 'Tim', 'start': 11, 'end': 14}
{'entity': 'I-PER', 'score': 0.9997162, 'index': 5, 'word': 'Cook', 'start': 15, 'end': 19}
{'entity': 'B-ORG', 'score': 0.9992311, 'index': 13, 'word': 'Apple', 'start': 36, 'end': 41}
{'entity': 'B-ORG', 'score': 0.9992084, 'index': 15, 'word': 'Apple', 'start': 43, 'end': 48}
{'entity': 'B-LOC', 'score': 0.99883693, 'index': 23, 'word': 'US', 'start': 78, 'end': 80}


2.**Text Summarization**

In [ ]:
from transformers import pipeline

Loading the model

In [ ]:
model = "facebook/bart-large-cnn"

In [ ]:
summarizer = pipeline("summarization", model=model)

In [ ]:
ARTICLE = """The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris.
Its base is square, measuring 125 metres (410 ft) on each side.
During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world,
a title it held for 41 years until the Chrysler Building in New York City was finished in 1930.
It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft).
Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
"""

Setting max and min lenght

In [ ]:
results = summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)

In [ ]:
print(results[0]["summary_text"])

The Eiffel Tower in Paris is the tallest structure in the world. It is now taller than the Chrysler Building in New York City. The tower is about the same height as a football pitch.


In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
raw_datasets["train"][0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,"Among the regions which saw the biggest losses were London and north-west England, which saw cuts of more than £12.3m.\nSports stars and charities said they were concerned cutting facilities was ""short-termism"" that could impact on communities' health and fitness levels.\nThe government said it was investing in grassroots sport.\nSome of the biggest cuts occurred in the North West, where Liverpool City Council closed Woolton Swimming Pool, saving more than £3m.\nIn the West Midlands, which saw £9.6m of cuts, the region's only 50m pool - in Coventry - was among the facilities to face the axe.\nAnd in London, where budgets were cut by £8.8m, Mornington Crescent Sports Centre in Camden was among the facilities to close.\nIn other regions, Sheffield lost the Don Valley Stadium, where Olympic heptathlon champion Jessica Ennis-Hill had trained, while Newcastle-upon-Tyne saw the closure of its City Pool in 2013.\nDavid Moorcroft, the Commonwealth Games gold medallist and former chief executive of UK Athletics, said: ""In times of cutbacks to public services, rightly or wrongly, sport and leisure is one of the first things to get cut.\n""It's really unfortunate because the health and happiness of the nation and communities is based around being able to access facilities that encourage people to take physical activity.\n""Ultimately, if we are trying to reduce obesity among young people, you can't really have clubs and volunteers doing all that work. Once a facility is lost, it's gone forever. When you come out of recession, it's very difficult to rebuild it.""\nEmma Boggis, chief executive of the Sport and Recreation Alliance which represents sports governing bodies in the UK, said she had ""some sympathy"" with local authorities ""and the extreme financial pressures they are under""\n""But reducing investment in sport and in leisure facilities is storing up problems for the longer-term,"" she said.\n""Limiting access to leisure facilities will result in greater inactivity and bigger costs to the NHS in terms of tackling conditions like diabetes, cardiovascular disease and depression.""\nThe Department for Culture, Media and Sport said £1bn of public money had been invested into grassroots sport through Sport England.\nDavid Sparks, who chairs the Local Government Association, said councils had had ""little choice"" but to squeeze budgets.\n""The reality is that, within a few years, well over half of the council tax everyone pays will have to be spent on social care,"" he said.\n""With demand on these life and death services continuing to rise and funding from central government continuing to fall, councils will have little choice.""","More than £42m has been axed from councils' sports and leisure budgets since 2010, a BBC survey has revealed.",31624412
1,"Coventry's Jim O'Brien had an effort tipped onto the bar before Jack Payne's low effort put the hosts ahead.\nBarnett ran onto a Stephen Hendrie through ball to slot home a second.\nGary Deegan was then dismissed for a second yellow on 51 minutes, but Barnett ensured victory from the penalty spot, after City keeper Reice Charles-Cook had fouled Will Atkinson.\nCoventry slip to fifth, six points off top spot in League One, after three games without a win, while Phil Brown's Southend climb to ninth, within two points of a play-off place.\nSky Blues boss Tony Mowbray told BBC Coventry & Warwickshire:\n""Obviously we're disappointed. We struggled from the start really.\n""We haven't got any excuses. We were beaten by a team who were very resilient. They dug in and didn't give us a minute on the ball.\n""We came here with a very experienced team. They defended resiliently, put a lot of men behind the ball and made life difficult for us.""","Tyrone Barnett scored twice as Southend comfortably beat Coventry City, despite playing almost 40 minutes with 10 men.",35330856
2,"Christopher Hodder set up a fake online profile and pretended to be a girl as he tricked his 

**3.Summarization**

In [ ]:
billsum = load_dataset("billsum", split="ca_test")

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
prefix = "summarize: "

In [ ]:
def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["text"]]
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
  labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_billsum["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_billsum["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=[metric_callback])

In [ ]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."